In [1]:
!pip install numpy


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
import pandas as pd
import openpyxl
import matplotlib.pyplot as plt
import numpy as np



In [3]:
# 데이터 디렉토리 경로
directory_o = r'C:\Users\ch.kang\Downloads\신청페이지 들어간사람\q_l'
directory_o2 = r'C:\Users\ch.kang\Downloads\신청페이지 들어간사람\ㅣ_l'

directory_x = r'C:\Users\ch.kang\Downloads\신청페이지 안들어간사람\전체'



# 모든 엑셀 파일의 경로를 가져오기
file_paths_o = [os.path.join(directory_o, file) for file in os.listdir(directory_o) if file.endswith('.xlsx')]
file_paths_o2 = [os.path.join(directory_o, file) for file in os.listdir(directory_o) if file.endswith('.xlsx')]
file_paths_x = [os.path.join(directory_x, file) for file in os.listdir(directory_x) if file.endswith('.xlsx')]



In [4]:
# 엑셀 파일을 읽어들여 데이터프레임으로 변환하여 리스트에 저장
dfs_o = []
for file_path_1 in file_paths_o:
    # 파일명에서 확장자를 제외한 부분 추출
    file_name = os.path.splitext(os.path.basename(file_path_1))[0]
    # 엑셀 파일을 읽어들여 데이터프레임으로 변환
    df_1 = pd.read_excel(file_path_1, dtype=str)
    # 'date' 열에 파일명에서 추출한 값을 추가
    df_1['date'] = file_name
    dfs_o.append(df_1)

dfs_o2 = []
for file_path_3 in file_paths_o:
    # 파일명에서 확장자를 제외한 부분 추출
    file_name = os.path.splitext(os.path.basename(file_path_3))[0]
    # 엑셀 파일을 읽어들여 데이터프레임으로 변환
    df_3 = pd.read_excel(file_path_3, dtype=str)
    # 'date' 열에 파일명에서 추출한 값을 추가
    df_3['date'] = file_name
    dfs_o2.append(df_3)


dfs_x = []
for file_path_2 in file_paths_x:
    # 파일명에서 확장자를 제외한 부분 추출
    file_name = os.path.splitext(os.path.basename(file_path_2))[0]
    # 엑셀 파일을 읽어들여 데이터프레임으로 변환
    df_2 = pd.read_excel(file_path_2, dtype=str)
    # 'date' 열에 파일명에서 추출한 값을 추가
    df_2['date'] = file_name
    dfs_x.append(df_2)



In [5]:
# 리스트에 있는 모든 데이터프레임을 하나로 합치기
merged_df_o = pd.concat(dfs_o, ignore_index=True)
print(merged_df_o.columns)
merged_df_x = pd.concat(dfs_x, ignore_index=True)
print(merged_df_x.columns)
merged_df_o2 = pd.concat(dfs_o2, ignore_index=True)
print(merged_df_o2.columns)

Index(['아이디', '스크롤여부', '클릭여부', '체류시간', 'date'], dtype='object')
Index(['아이디', '체류시간', '이탈유형', 'date'], dtype='object')
Index(['아이디', '스크롤여부', '클릭여부', '체류시간', 'date'], dtype='object')


In [6]:
merged_df_o["신청진입여부"] = "Y"
merged_df_o2["신청진입여부"] = "Y"
merged_df_x["신청진입여부"] = "N"
# Sort the DataFrame by '체류시간' in ascending order
merged_df_o['체류시간'] = pd.to_numeric(merged_df_o['체류시간'], errors='coerce')
merged_df_x['체류시간'] = pd.to_numeric(merged_df_x['체류시간'], errors='coerce')
merged_df_o2['체류시간'] = pd.to_numeric(merged_df_o2['체류시간'], errors='coerce')

df_sorted_o = merged_df_o.sort_values(by='체류시간', ascending=True).reset_index(drop=True)
df_sorted_x = merged_df_x.sort_values(by='체류시간', ascending=True).reset_index(drop=True)
df_sorted_o2 = merged_df_o2.sort_values(by='체류시간', ascending=True).reset_index(drop=True)

# Calculate the cumulative percentage for '체류시간'
df_sorted_o['누적%'] = (df_sorted_o.index + 1) / len(df_sorted_o) * 100
df_sorted_o['누적%'] = df_sorted_o['누적%'].round(1)
df_sorted_o

df_sorted_x['누적%'] = (df_sorted_x.index + 1) / len(df_sorted_x) * 100
df_sorted_x['누적%'] = df_sorted_x['누적%'].round(1)
df_sorted_x

df_sorted_o2['누적%'] = (df_sorted_o2.index + 1) / len(df_sorted_o2) * 100
df_sorted_o2['누적%'] = df_sorted_o2['누적%'].round(1)
df_sorted_o2

merged_df = pd.concat([df_sorted_o,df_sorted_o2, df_sorted_x], ignore_index=True)
print(len(df_sorted_o),len(df_sorted_o2),len(df_sorted_x), len(merged_df))


# 체류시간을 10초 간격으로 나누어 새로운 열 추가
merged_df['10초 구간'] = (merged_df['체류시간'] // 10) * 10
merged_df.columns


13 13 2430 2456


Index(['아이디', '스크롤여부', '클릭여부', '체류시간', 'date', '신청진입여부', '누적%', '이탈유형',
       '10초 구간'],
      dtype='object')

In [7]:
df=merged_df.copy()
len(df)

2456

In [8]:
def detect_outliers(data):
    """
    주어진 데이터에서 이상치를 판단하는 함수
    :param data: 이상치를 판단할 데이터 (시리즈 형태)
    :return: 이상치 여부를 나타내는 시리즈를 반환
    """
    # Z-점수 계산
    z_scores = np.abs((data - np.mean(data)) / np.std(data))
    # 이상치 판단
    outliers_mask = z_scores > 3
    return outliers_mask

# merged_df['체류시간'] 데이터를 사용하여 이상치 여부를 판단
outliers_mask = detect_outliers(df['체류시간'])

# 이상치 여부를 나타내는 열 추가
df['이상치'] = outliers_mask



In [9]:
df

,아이디,스크롤여부,클릭여부,체류시간,date,신청진입여부,누적%,이탈유형,10초 구간,이상치
0,1042732193.1712370545,N,N,2,20240408,Y,7.7,NaN,0,False
1,1819785872.1712371735,N,N,10,20240406,Y,15.4,NaN,10,False
2,1653590084.1712352132,N,N,15,20240406,Y,23.1,NaN,10,False
3,2084490160.1712804830,Y,N,17,20240411,Y,30.8,NaN,10,False
4,782439199.1712662749,Y,N,17,20240411,Y,38.5,NaN,10,False
...,...,...,...,...,...,...,...,...,...,...
2451,1401617923.1710214328,NaN,NaN,13445,20240410,N,99.8,out,13440,True
2452,1765429207.1710411943,NaN,NaN,22564,20240406,N,99.9,out,22560,True
2453,1657660699.1712372882,NaN,NaN,26918,20240406,N,99.9,in,26910,True
2454,1534323344.1712513206,NaN,NaN,28597,20240408,N,100.0,out,28590,True


In [10]:
# Define the binary mapping with all expected values
binary_mapping = {'Y': 1, 'N': 0, 'in': 2, 'out': 3}

# Apply the mapping while filling NaN values
df['스크롤여부'] = df['스크롤여부'].map(binary_mapping).fillna(-1)
df['클릭여부'] = df['클릭여부'].map(binary_mapping).fillna(-1)
df['신청진입여부'] = df['신청진입여부'].map(binary_mapping).fillna(-1)
df['이탈유형'] = df['이탈유형'].map(binary_mapping).fillna(-1)


In [11]:
df

,아이디,스크롤여부,클릭여부,체류시간,date,신청진입여부,누적%,이탈유형,10초 구간,이상치
0,1042732193.1712370545,0.0,0.0,2,20240408,1,7.7,-1.0,0,False
1,1819785872.1712371735,0.0,0.0,10,20240406,1,15.4,-1.0,10,False
2,1653590084.1712352132,0.0,0.0,15,20240406,1,23.1,-1.0,10,False
3,2084490160.1712804830,1.0,0.0,17,20240411,1,30.8,-1.0,10,False
4,782439199.1712662749,1.0,0.0,17,20240411,1,38.5,-1.0,10,False
...,...,...,...,...,...,...,...,...,...,...
2451,1401617923.1710214328,-1.0,-1.0,13445,20240410,0,99.8,3.0,13440,True
2452,1765429207.1710411943,-1.0,-1.0,22564,20240406,0,99.9,3.0,22560,True
2453,1657660699.1712372882,-1.0,-1.0,26918,20240406,0,99.9,2.0,26910,True
2454,1534323344.1712513206,-1.0,-1.0,28597,20240408,0,100.0,3.0,28590,True


In [12]:
# Convert columns to appropriate data types
df['체류시간'] = pd.to_numeric(df['체류시간'], errors='coerce')
df['10초 구간'] = pd.to_numeric(df['10초 구간'], errors='coerce')

# Calculate the correlation matrix
correlation_matrix = df.corr()

print("""0.9 이상 또는 -0.9 이하: 매우 강한 상관관계
0.7 이상 또는 -0.7 이하: 강한 상관관계
0.5 이상 또는 -0.5 이하: 중간 정도의 상관관계
0.3 이상 또는 -0.3 이하: 약한 상관관계
0.1 이상 또는 -0.1 이하: 매우 약한 상관관계
0: 상관관계가 없음""")
      
correlation_matrix


0.9 이상 또는 -0.9 이하: 매우 강한 상관관계
0.7 이상 또는 -0.7 이하: 강한 상관관계
0.5 이상 또는 -0.5 이하: 중간 정도의 상관관계
0.3 이상 또는 -0.3 이하: 약한 상관관계
0.1 이상 또는 -0.1 이하: 매우 약한 상관관계
0: 상관관계가 없음


,아이디,스크롤여부,클릭여부,체류시간,date,신청진입여부,누적%,이탈유형,10초 구간,이상치
아이디,1.000000,0.025407,0.032407,0.011993,0.016418,0.025620,0.018793,-0.028606,0.011979,0.014644
스크롤여부,0.025407,1.000000,0.948170,0.276700,-0.019260,0.964407,0.032396,-0.832693,0.276632,0.367668
클릭여부,0.032407,0.948170,1.000000,0.239247,-0.018687,0.970426,0.020799,-0.837891,0.239203,0.387928
체류시간,0.011993,0.276700,0.239247,1.000000,-0.032244,0.241490,0.141025,-0.244140,0.999999,0.869717
date,0.016418,-0.019260,-0.018687,-0.032244,1.000000,-0.026507,-0.081997,0.020091,-0.032257,-0.024658
신청진입여부,0.025620,0.964407,0.970426,0.241490,-0.026507,1.000000,0.013562,-0.863426,0.241436,0.321424
누적%,0.018793,0.032396,0.020799,0.141025,-0.081997,0.013562,1.000000,-0.134813,0.139889,0.117052
이탈유형,-0.028606,-0.832693,-0.837891,-0.244140,0.020091,-0.863426,-0.134813,1.000000,-0.243889,-0.296536
10초 구간,0.011979,0.276632,0.239203,0.999999,-0.032257,0.241436,0.139889,-0.243889,1.000000,0.869729
이상치,0.014644,0.367668,0.387928,0.869717,-0.024658,0.321424,0.117052,-0.296536,0.869729,1.000000


In [13]:
# Define the threshold for strong correlation (absolute value of 0.7 or more)
threshold = 0.7

# Create a mask to identify correlations above the threshold
correlation_mask = correlation_matrix.abs() >= threshold

# Filter out correlations below the threshold
strong_correlations = correlation_matrix[correlation_mask].fillna('')

strong_correlations


,아이디,스크롤여부,클릭여부,체류시간,date,신청진입여부,누적%,이탈유형,10초 구간,이상치
아이디,1.0,,,,,,,,,
스크롤여부,,1.0,0.94817,,,0.964407,,-0.832693,,
클릭여부,,0.94817,1.0,,,0.970426,,-0.837891,,
체류시간,,,,1.0,,,,,0.999999,0.869717
date,,,,,1.0,,,,,
신청진입여부,,0.964407,0.970426,,,1.0,,-0.863426,,
누적%,,,,,,,1.0,,,
이탈유형,,-0.832693,-0.837891,,,-0.863426,,1.0,,
10초 구간,,,,0.999999,,,,,1.0,0.869729
이상치,,,,0.869717,,,,,0.869729,1.0


In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

# Prepare the feature set (independent variables) and target variable (dependent variable)
# List of columns to retain as features
selected_features = ['스크롤여부', '체류시간', '누적%']
# Select only the specified columns as features
features = df[selected_features]
target = df['신청진입여부']

# Handle missing values
features_filled = features.fillna(-1)

# One-hot encoding for categorical features
features_encoded = pd.get_dummies(features_filled, drop_first=True)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features_encoded, target, test_size=0.3, random_state=42)

# Standardize the feature data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialize and train the logistic regression model
model = LogisticRegression(max_iter=1000, random_state=42)
model.fit(X_train_scaled, y_train)

# Get the feature importance (coefficients) from the trained model
coefficients = model.coef_[0]

# Create a DataFrame to present the feature importance
feature_importance = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': coefficients
}).sort_values(by='Importance', ascending=False)

# Display the feature importance
print(feature_importance)


  Feature  Importance
0   스크롤여부    2.067802
1    체류시간   -0.006103
2     누적%   -0.257146


In [17]:
# Separate features based on the coefficient sign
positive_coefficients = feature_importance[feature_importance['Importance'] > 0]
negative_coefficients = feature_importance[feature_importance['Importance'] < 0]

# Display results
print("Features that increase the likelihood of '신청진입여부' being 1:")
print(positive_coefficients)

print("\nFeatures that increase the likelihood of '신청진입여부' being 0:")
print(negative_coefficients)

print("""Features that Increase the Likelihood of '신청진입여부' Being 1:
스크롤여부
스크롤여부 변수의 회귀 계수가 양수이므로, 스크롤을 사용한 방문자는 신청진입여부가 1이 될 가능성이 높습니다. 즉, 스크롤을 사용한 방문자는 특정 페이지에 진입할 가능성이 증가합니다.
Features that Increase the Likelihood of '신청진입여부' Being 0:
체류시간 
체류시간 변수의 회귀 계수가 음수이므로, 체류 시간이 증가하면 신청진입여부가 0이 될 가능성이 높아집니다. 즉, 페이지에 오래 머물수록 신청 진입을 하지 않을 가능성이 높아집니다.
누적% 
누적% 변수의 회귀 계수도 음수이므로, 누적 백분율이 높아질수록 신청진입여부가 0이 될 가능성이 높습니다. 이는 페이지 방문을 거듭하며 누적된 방문 경험이 신청 진입을 하지 않을 가능성을 높인다는 것을 의미할 수 있습니다.""")

Features that increase the likelihood of '신청진입여부' being 1:
  Feature  Importance
0   스크롤여부    2.067802

Features that increase the likelihood of '신청진입여부' being 0:
  Feature  Importance
1    체류시간   -0.006103
2     누적%   -0.257146
Features that Increase the Likelihood of '신청진입여부' Being 1:
스크롤여부
스크롤여부 변수의 회귀 계수가 양수이므로, 스크롤을 사용한 방문자는 신청진입여부가 1이 될 가능성이 높습니다. 즉, 스크롤을 사용한 방문자는 특정 페이지에 진입할 가능성이 증가합니다.
Features that Increase the Likelihood of '신청진입여부' Being 0:
체류시간 
체류시간 변수의 회귀 계수가 음수이므로, 체류 시간이 증가하면 신청진입여부가 0이 될 가능성이 높아집니다. 즉, 페이지에 오래 머물수록 신청 진입을 하지 않을 가능성이 높아집니다.
누적% 
누적% 변수의 회귀 계수도 음수이므로, 누적 백분율이 높아질수록 신청진입여부가 0이 될 가능성이 높습니다. 이는 페이지 방문을 거듭하며 누적된 방문 경험이 신청 진입을 하지 않을 가능성을 높인다는 것을 의미할 수 있습니다.


In [16]:
import pandas as pd

# Create a path or filename for the Excel file
excel_path = 'data_analysis_results_ql.xlsx'

# Use ExcelWriter to write each DataFrame into different sheets
with pd.ExcelWriter(excel_path) as writer:
    df.to_excel(writer, sheet_name='Original Data', index=False)  # Original data
    correlation_matrix.to_excel(writer, sheet_name='Correlation Matrix')  # Correlation matrix
    feature_importance.to_excel(writer, sheet_name='Feature Importance', index=False)  # Feature importance
